<a href="https://colab.research.google.com/github/anothermartz/Easy-Wav2Lip/blob/main/Wav2Lip_HQ_working_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Make sure to click 👆 that button to copy it your own Google Drive first!

# Wav2Lip-HQ made easy!

* Using this code: https://github.com/GucciFlipFlops1917/wav2lip-hq-updated-ESRGAN

* Original Wav2Lip Github: https://github.com/Rudrabha/Wav2Lip
* Depreciated Wav2LipHQ Github: https://github.com/Markfryazino/wav2lip-hq

This colab was made by anothermartz - I may offer some support in this discord:<br>
Invite: https://discord.gg/FNZR9ETwKY<br>
Wav2Lip channel: https://discord.com/channels/667279414681272320/1076077584330280991

# How to format your files for this colab:
Video files:
* Must be .mp4
* Must have a face in all frames
* Small file in every way (try <720p, <60 seconds, <10mbps <b></b> bitrate etc. - Bigger files may work but are usually the reason it fails)
* Bear in mind, a 720p file at 10mbps will take about 45 seconds for each second of video to process!

Audio files:
* Ideally just encode it into your .mp4 files
* <b>OR</b>
* Must be .wav
* Same folder and filename as video files eg: File1.wav, File2.wav, File 3.wav

Technically more formats are supported with Wav2Lip, but I've only coded it to support .mp4 and .wav for now

In [ ]:
#@title <h1>Step 1: Setup "wav2lip-hq-updated-ESRGAN"</h1> Such a catchy title
#@markdown 1. 👈 Click that little circle play button.
#@markdown 2. If your files are on Google Drive (recommended), link your Google Drive when prompted
#@markdown <br><br> Alternatively, say "no thanks" and click the folder icon to the far left, right click and upload your files there.<br>If not using Google Drive, you may lose all processed files if not manually downloaded.

import sys
import tensorflow as tf
import re

if not tf.test.gpu_device_name():
    sys.exit('No GPU found. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')
else:
    print('GPU found, requesting google drive permission')

try:
  from google.colab import drive
  drive.mount('/content/drive')
except Exception as e:
  print("Google Drive not linked")

try:
  !git clone https://github.com/GucciFlipFlops1917/wav2lip-hq-updated-ESRGAN.git
  %cd wav2lip-hq-updated-ESRGAN
  !pip3 install -r requirements.txt
  !pip3 install gdown
  !pip install gfpgan
  !pip install realesrgan

  #download required files
  !wget "https://github.com/GucciFlipFlops1917/wav2lip-hq-updated-ESRGAN/releases/download/v0.0.1/face_segmentation.pth" -O "/content/wav2lip-hq-updated-ESRGAN/checkpoints/face_segmentation.pth"
  !wget "https://download.pytorch.org/models/resnet18-5c106cde.pth" -O "/root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth"
  !wget "https://github.com/GucciFlipFlops1917/wav2lip-hq-updated-ESRGAN/releases/download/v0.0.1/4x_BigFace_v3_Clear.pth" -O "/content/wav2lip-hq-updated-ESRGAN/weights/4x_BigFace_v3_Clear.pth"
  !wget "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" -O "/content/wav2lip-hq-updated-ESRGAN/gfpgan/weights/detection_Resnet50_Final.pth"
  !wget "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" -O "/content/wav2lip-hq-updated-ESRGAN/gfpgan/weights/parsing_parsenet.pth"
  !wget "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth" -O "/usr/local/lib/python3.9/dist-packages/gfpgan/weights/GFPGANv1.4.pth"
  !wget "https://github.com/GucciFlipFlops1917/wav2lip-hq-updated-ESRGAN/releases/download/v0.0.1/wav2lip_gan.pth" -O "/content/wav2lip-hq-updated-ESRGAN/checkpoints/wav2lip_gan.pth"
  !wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "face_detection/detection/sfd/s3fd.pth"

  #import fixed audio.py
  import gdown
  gdown.download("https://drive.google.com/uc?id=1YtY09TAXghJS8zCnjDw3JwwcYJdd_aZv","/content/wav2lip-hq-updated-ESRGAN/audio.py")

  import os
  from IPython.display import HTML, clear_output
  from base64 import b64encode
  def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

  clear_output()
  print("Installation complete, move to Step 2!")
except Exception as e:
  print("Whoops, what went wrong?")

In [ ]:
#@title <h1>Step 2: Select file</h1>
#@markdown <h1>Select video:</h1>

input_path = "" #@param {type:"string"}
#@markdown >👆 Look for the folder icon at the left edge of colab and find your video, right click it, copy path & paste it above
# Check if file exists
# Check if path ends with .mp4
if not input_path.endswith(".mp4"):
    sys.exit('File is not .mp4')
else:
  if not os.path.exists(input_path):
      sys.exit(f'Could not find file: {input_path}')
      raise SystemExit(0)
  else:
      # Extract filename
      filename_pattern = r"[^\/]+(?=\.\w+$)"
      filename = re.search(filename_pattern, input_path).group()

      # Extract folder
      folder_pattern = r"^(.*\/)[^\/]+$"
      folder = re.search(folder_pattern, input_path).group(1)

      # Extract filenumber for batch processing (coming in the future)
      filenumber_pattern = r"\d+$"
      filenumber_match = re.search(filenumber_pattern, filename)
      if filenumber_match:
          filenumber = filenumber_match.group()
          filenamenonumber = re.sub(filenumber_pattern, "", filename)
      else:
          filenumber = None
          filenamenonumber = filename

#Batch_Process= True #@param {type:"boolean"}
##@markdown >Disable if you just want to process one video (good for testing or fixing padding).
output_suffix = "_Wav2LipHQ" #@param {type:"string"}
#@markdown >This adds a suffix to your output files so that they don't overwite your originals
output_file = folder + filename + output_suffix + ".mp4"
output_filename = filename + output_suffix + ".mp4"
input_audio = folder + filename + ".wav"
input_wav = folder + filename + ".wav"
audio_file = r"[^\/]+$"
input_audiofile = re.search(audio_file, input_audio).group()

# Print results
input_video = folder + filenamenonumber + filenumber + ".mp4"
print("video:" , input_video)
showVideo(input_video)

try:
  input_audio = folder + filenamenonumber + filenumber + ".wav"
  if os.path.isfile(input_audio):
    print("audio found:" , input_audio)
  else:
    print(f"{input_audiofile} not found in same folder - attempting to take audio from video file...")
except Exception as e:
  print("Unable to extract audio from video file, use a seperate .wav file instead!")

if os.path.isfile('/content/inputs/' + input_audiofile):
    os.remove('/content/inputs/' + input_audiofile)

if os.path.isfile(output_file):
    os.remove(output_file)

if os.path.isfile(input_audio):
  # Load audio with specified sampling rate
  print(input_audiofile + " found")
  display(Audio(input_audio))
else:
  import librosa
  audio, sr = librosa.load(input_video, sr=None)
  import soundfile as sf
  sf.write('/content/sample_data/input_audio.wav', audio, sr, format='wav')
  input_audio = '/content/sample_data/input_audio.wav'
  clear_output()
  print("video:" , input_video)
  showVideo(input_video)
  print("Successfully taken audio from video")

from numpy.lib import stride_tricks
#@markdown <h1><br>Tweak padding:</h1> (Up, Down, Left, Right) <br>
#@markdown <b><br>Lower values typically look better on the mouth but can cause hard lines at the edges of the face (typically on the chin)</b>
U = 0 #@param {type:"slider", min:0, max:100, step:1}
D =  0 #@param {type:"slider", min:0, max:100, step:1}
L =  0 #@param {type:"slider", min:0, max:100, step:1}
R =  0 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Lower the output resolution for quicker rendering and better hiding of artifacts, at the cost of worse overall image quality:
ResolutionScale =  1 #@param {type:"slider", min:0.25, max:1, step:0.25}
#@markdown Disable face detection smoothing which may fix artifacts, I'm not aware of any downsides to this:
nosmooth = True #@param {type:"boolean"}

rescaleFactor = str(round(1 // ResolutionScale))

pad_up = str(round(U * ResolutionScale))
pad_down = str(round(D * ResolutionScale))
pad_left = str(round(L * ResolutionScale))
pad_right = str(round(R * ResolutionScale))

temp_output = "results/output.mp4"

if os.path.isfile(temp_output):
    os.remove(temp_output)

if nosmooth == False:
  !python inference.py \
    --face {input_path} \
    --audio {input_audio} \
    --outfile {temp_output} \
    --pads $pad_up $pad_down $pad_left $pad_right \
    --resize_factor $rescaleFactor \
    --enhance_face 'gfpgan'
else:
  !python inference.py \
    --face {input_path} \
    --audio {input_audio} \
    --outfile {temp_output} \
    --pads $pad_up $pad_down $pad_left $pad_right \
    --resize_factor $rescaleFactor \
    --enhance_face 'gfpgan' \
    --nosmooth

if os.path.isfile(temp_output):
  os.rename(temp_output, output_filename )
  showVideo(output_filename)
  !cp {output_filename} {folder}
  clear_output()
  print(f"{filename}.mp4 successfully lip synched!, copying to here:")
  print(output_file)
  showVideo(output_filename)
else:
  print(f"Processing failed! :(")